In [2]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 1.6 MB/s eta 0:00:0000:0100:01


In [5]:
import requests  # makes http requests
from bs4 import BeautifulSoup  # parses html content
import pandas as pd  # data manipulation and storage
import time  # delays between requests
import random  # adds variability in request delays
import re  # pattern matching in text
import mysql.connector

# amazon url for data engineering books
URL = "https://www.packtpub.com/en-us/search?q=data%20engineering%20books&country=us&language=en"

# user-agent list to avoid detection by amazon's anti-scraping measures
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
]

# headers to mimic a real browser request
HEADERS = {
    "User-Agent": random.choice(USER_AGENTS),  # randomly selects a user-agent to reduce detection risk
    "Accept-Language": "en-US,en;q=0.5"  # specifies the preferred language for the response
}

def get_publication_date_from_product_page(book_url):
    """fetches the publication date from a book's product page."""
    try:
        product_response = requests.get(book_url, headers=HEADERS)  # sends request to the book's product page
        if product_response.status_code == 200:  # checks if request was successful
            product_soup = BeautifulSoup(product_response.text, "html.parser")  # parses html content
            possible_locations = [  # defines possible locations where publication date might be found
                "#detailBullets_feature_div",
                "#productDetailsTable",
                "#prodDetails"
            ]
            for location in possible_locations:
                details = product_soup.select_one(location)  # selects the first matching html element
                if details:
                    detail_text = details.get_text(strip=True)  # extracts text and removes extra spaces
                    date_match = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}", detail_text)
                    if date_match:
                        return date_match.group()  # returns the matched publication date

            year_match = re.search(r"(19|20)\d{2}", product_soup.text)  # searches for a 4-digit year in the text
            if year_match:
                return year_match.group()  # returns the matched year
    except Exception as e:
        print(f"error fetching product page: {e}")  # prints error message if fetching fails
    return None  # returns none if no publication date is found

response = requests.get(URL, headers=HEADERS)  # sends a request to amazon

if response.status_code == 200:  # checks if request was successful
    soup = BeautifulSoup(response.text, "html.parser")  # parses html content
    books = soup.find_all("div", {"data-component-type": "s-search-result"})  # finds all book listings

    # lists to store extracted book data
    titles, authors, pub_dates, ratings, prices = [], [], [], [], []

    for book in books[:50]:  # loops through the first 50 book listings
        title_tag = book.find("h2", class_="a-size-base-plus a-spacing-none a-color-base a-text-normal")  # finds book title
        title = title_tag.text.strip() if title_tag else "unknown"  # extracts title text or assigns "unknown"

        author_tag = book.find("div", class_="a-row a-size-base a-color-secondary")  # finds author information
        author_links = author_tag.find_all("a") if author_tag else []  # gets all author links if present
        author = ", ".join([a.text.strip() for a in author_links]) if author_links else "unknown"  # extracts author names

        pub_date = None  # initializes publication date variable
        date_spans = book.find_all("span", class_="a-size-base a-color-secondary")  # finds potential publication date elements
        for span in date_spans:
            span_text = span.get_text(strip=True)  # extracts text content
            date_match = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}", span_text)
            if date_match:
                pub_date = date_match.group()  # assigns found publication date
                break

        if pub_date is None:
            year_match = re.search(r"(19|20)\d{2}", book.text)  # searches for a 4-digit year
            if year_match:
                pub_date = year_match.group()  # assigns found year

        rating_tag = book.find("span", class_="a-icon-alt")  # finds book rating
        rating = rating_tag.text.strip().split()[0] if rating_tag else "no rating"  # extracts rating value

        price_tag = book.find("span", class_="a-price-whole") or book.find("span", class_="a-offscreen")  # finds price
        price = price_tag.text.strip() if price_tag else "not available"  # extracts price or assigns "not available"

        book_url_tag = book.find("a", class_="a-link-normal s-no-outline")  # finds book url
        book_url = "https://www.amazon.com" + book_url_tag["href"] if book_url_tag else None  # constructs full url

        if pub_date is None and book_url:
            pub_date = get_publication_date_from_product_page(book_url)  # gets publication date from product page if missing

        # appends extracted data to lists
        titles.append(title)
        authors.append(author)
        pub_dates.append(pub_date)
        ratings.append(rating)
        prices.append(price)
        time.sleep(random.uniform(1, 3))  # adds random delay to avoid detection

    # creates a pandas dataframe with collected book data
    df = pd.DataFrame({
        "Title": titles,
        "Author": authors,
        "Publication Date": pub_dates,
        "Rating": ratings,
        "Price": prices
    })

    # saves data to csv file
    df.to_csv("amazon_data_engineering_books1.csv", index=False)
    print("Data has been saved to 'data_engineering_books.csv'")

    try:
        # establishes database connection
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Aliza.123",
            database="data_engineering_books"
        )
        cursor = conn.cursor()

        for index, row in df.iterrows():  # iterates through dataframe rows
            if all(pd.notnull(val) and val != "unknown" for val in row):  # checks if row data is valid
                cursor.execute("""
                    INSERT INTO books (title, author, publication_date, rating, price)
                    VALUES (%s, %s, %s, %s, %s)
                """, (row['Title'], row['Author'], row['Publication Date'], row['Rating'], row['Price']))  # inserts data
        conn.commit()  # commits transaction

        # query that extract only 3 columns from the table and sorts table based on rating column
        query = """
            SELECT title, author, price
            FROM books
            ORDER BY rating ASC;
        """
        cursor.execute(query)
        results = cursor.fetchall()
        df_result = pd.DataFrame(results, columns=["Title", "Author", "Price", "Rating"])
        print(df_result)

    # handles database errors
    except mysql.connector.Error as db_error:
        print(f"database error: {db_error}")

    finally:
        cursor.close()
        # closes connection to database
        conn.close()
        print("\nThese are the top-rated data engineering books our shoppers love ♡. \nHappy reads!")

else:
    # error handling message for user if parsing fails
    print("ⓘ Failed to retrieve webpage. Amazon may have blocked the request. :(")


Data has been saved to 'data_engineering_books.csv'
database error: 1049 (42000): Unknown database 'data_engineering_books'


NameError: name 'cursor' is not defined

In [7]:
import mysql.connector
import pandas as pd

# Ensure database connection and creation
def create_database():
    """Ensures that the MySQL database exists."""
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aliza.123"
    )
    cursor = conn.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS data_engineering_books")
    conn.commit()
    cursor.close()
    conn.close()

# Ensure table creation
def create_table():
    """Ensures that the 'books' table exists with the correct schema."""
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aliza.123",
        database="data_engineering_books"
    )
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS books (
            id INT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(255),
            author VARCHAR(255),
            publication_date VARCHAR(50),
            rating VARCHAR(10),
            price VARCHAR(50)
        )
    """)
    
    conn.commit()
    cursor.close()
    conn.close()

# Load data from the CSV file
def load_data():
    """Loads scraped data from CSV and inserts it into MySQL."""
    df = pd.read_csv("amazon_data_engineering_books.csv")

    conn = None
    cursor = None

    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Aliza.123",
            database="data_engineering_books"
        )
        cursor = conn.cursor()

        # Insert data into MySQL
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT INTO books (title, author, publication_date, rating, price)
                VALUES (%s, %s, %s, %s, %s)
            """, (row['Title'], row['Author'], row['Publication Date'], row['Rating'], row['Price']))
        
        conn.commit()
        print("Data successfully inserted into MySQL!")

    except mysql.connector.Error as db_error:
        print(f"Database error: {db_error}")

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Extract and sort data
def query_books():
    """Executes the SQL query to extract and sort books based on rating."""
    conn = mysql.connector.connect(
        host="localhost",
        user="root",

Database error: 1054 (42S22): Unknown column 'nan' in 'field list'

Top-rated Data Engineering Books:

Empty DataFrame
Columns: [Title, Author, Price, Publication Date]
Index: []


In [9]:
import mysql.connector
import pandas as pd

# Ensure database connection and creation
def create_database():
    """Ensures that the MySQL database exists."""
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aliza.123"
    )
    cursor = conn.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS data_engineering_books")
    conn.commit()
    cursor.close()
    conn.close()

# Ensure table creation
def create_table():
    """Ensures that the 'books' table exists with the correct schema."""
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aliza.123",
        database="data_engineering_books"
    )
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS books (
            id INT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(255),
            author VARCHAR(255),
            publication_date VARCHAR(50),
            rating VARCHAR(10),
            price VARCHAR(50)
        )
    """)
    
    conn.commit()
    cursor.close()
    conn.close()

# Load data from the CSV file and handle NaN values
def load_data():
    """Loads scraped data from CSV, handles missing values, and inserts into MySQL."""
    df = pd.read_csv("amazon_data_engineering_books.csv")

    # Fill NaN values with "Unknown" for text fields, "Not Available" for price/publication date
    df.fillna({
        'Title': 'Unknown',
        'Author': 'Unknown',
        'Publication Date': 'Not Available',
        'Rating': 'No Rating',
        'Price': 'Not Available'
    }, inplace=True)

    conn = None
    cursor = None

    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Aliza.123",
            database="data_engineering_books"
        )
        cursor = conn.cursor()

        # Insert data into MySQL
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT INTO books (title, author, publication_date, rating, price)
                VALUES (%s, %s, %s, %s, %s)
            """, (row['Title'], row['Author'], row['Publication Date'], row['Rating'], row['Price']))
        
        conn.commit()
        print("Data successfully inserted into MySQL!")

    except mysql.connector.Error as db_error:
        print(f"Database error: {db_error}")

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Extract and sort data
def query_books():
    """Executes the SQL query to extract and sort books based on rating."""
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aliza.123",
        database="data_engineering_books"
    )
    cursor = conn.cursor()

    query = """
        SELECT title, author, price, publication_date
        FROM books
        ORDER BY rating ASC;
    """
    cursor.execute(query)
    results = cursor.fetchall()

    df_result = pd.DataFrame(results, columns=["Title", "Author", "Price", "Publication Date"])
    
    print("\nTop-rated Data Engineering Books:\n")
    print(df_result)

    cursor.close()
    conn.close()

# Execute functions
create_database()
create_table()
load_data()
query_books()


Data successfully inserted into MySQL!

Top-rated Data Engineering Books:

                                                Title  \
0   "Cracking the Data Engineering Interview: Land...   
1   "Cracking the Data Engineering Interview: Land...   
2   "Data Engineering with Python: Work with massi...   
3   "Data Engineering with AWS: Acquire the skills...   
4   "Data Engineering with AWS: Acquire the skills...   
5   "Data Engineering with Alteryx: Helping data e...   
6   "Data Engineering with Databricks Cookbook: Bu...   
7                         "Data Engineering on Azure"   
8   "AI Engineering: Building Applications with Fo...   
9   "Data Pipelines Pocket Reference: Moving and P...   
10  "Storytelling with Data: A Data Visualization ...   
11  "Data Labeling in Machine Learning with Python...   
12  "Fundamentals of Data Engineering: Plan and Bu...   
13  "Building Data Science Applications with FastA...   
14                       "Snowflake Data Engineering"   
15  "Data Eng